In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace_bryanq.bronze.bronze_volume;
CREATE VOLUME IF NOT EXISTS workspace_bryanq.silver.silver_volume;
CREATE VOLUME IF NOT EXISTS workspace_bryanq.gold.gold_volume;

In [0]:
dbutils.widgets.text("p_source", "")
p_source = dbutils.widgets.get("p_source")
display(p_source)

## **INCREMENTAL DATA INGESTION**

In [0]:
df = spark.readStream.format("cloudFiles")\
                    .option("cloudFiles.format", "csv")\
                    .option("cloudFiles.schemaLocation", f"/Volumes/workspace_bryanq/bronze/bronze_volume/{p_source}/checkpoint")\
                    .option("cloudFiles.schemaEvolutionMode", "rescue")\
                    .load(f"/Volumes/workspace_bryanq/raw/raw_volume/raw_data/{p_source}/")

In [0]:
df.writeStream.format("delta")\
              .outputMode("append")\
              .trigger(once = True)\
              .option("checkpointLocation", f"/Volumes/workspace_bryanq/bronze/bronze_volume/{p_source}/checkpoint")\
              .option("path", f"/Volumes/workspace_bryanq/bronze/bronze_volume/{p_source}/data")\
              .start()

In [0]:
query = f"""
        SELECT * FROM delta.`/Volumes/workspace_bryanq/bronze/bronze_volume/{p_source}/data`
        """

df = spark.sql(query)
display(df)